In [1]:
#Import the following Libraries for Twitter Streaming and Analysis
# If any library fails to import, make sure that you have installed the libraries properly
import tweepy
import twitter
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
from tweepy import OAuthHandler
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import operator 
import json
from collections import Counter
from nltk import bigrams
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from collections import defaultdict
import sys
import os

# To run this code, first edit config.py with your configuration directory and then import the config to be called leater on
# This config.py will contain the follwoing information for a specific user

# consumer_key = ''
# consumer_secret = ''
# access_token = ''
# access_secret = ''

import config

In [2]:
import os
os.getcwd()

'C:\\Users\\Mridul\\Mining-the-Social-Web-2nd-Edition-master\\ipynb'

In [2]:
#Using config.py file imported earlier, authenticate and initialize the api's
#Below i have used two APIs "tweepy and Twitter", and initialized both of them
auth = OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_secret)
api = tweepy.API(auth)

oauth = twitter.oauth.OAuth(config.access_token, config.access_secret,config.consumer_key, config.consumer_secret)
twitter_api = twitter.Twitter(auth=oauth)

In [3]:
# Choose a file name for saving the Tweets
fname_save = 'CARvsDAL6.json'
#Choose a file name for analyzing the tweets
fname = 'CARvsDAL4.json'

In [ ]:
#Create a MyListener class and call the funtion on_data to collect a live stream of data 
# with a specific KeyWord for example #CARvsDAL

class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open(fname_save, 'a+') as f:
                f.write(data)
                f.close()
                
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#CARvsDAL'])

In [4]:
#Function to pretty print the JSON file we have collected in the previous function call
#this function is just for testing purposes to see that the JSON data is loaded from the file

with open(fname, 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    print(json.dumps(tweet, indent=4)) # pretty-print
    f.close()

{
    "contributors": null, 
    "truncated": false, 
    "text": "RT @Panthers: Who wants to name this dance because we have no idea. \n\n#Panthers up 30-6! #CARvsDAL https://t.co/nnAN6mvJAM", 
    "is_quote_status": false, 
    "in_reply_to_status_id": null, 
    "id": 670029255082942464, 
    "favorite_count": 0, 
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 
    "retweeted": false, 
    "coordinates": null, 
    "timestamp_ms": "1448582397610", 
    "entities": {
        "user_mentions": [
            {
                "id": 56443153, 
                "indices": [
                    3, 
                    12
                ], 
                "id_str": "56443153", 
                "screen_name": "Panthers", 
                "name": "Carolina Panthers"
            }
        ], 
        "symbols": [], 
        "hashtags": [
            {
                "indices": [
                    70, 
                    79
         

In [5]:
# Function to Pre-process the tweets
# @-mentions, emoticons, URLs and #hash-tags are now preserved as individual tokens
# We try to capture some emoticons, HTML tags, Twitter @usernames (@-mentions), 
# Twitter #hashtags, URLs, numbers, words with and without dashes and apostrophes, and finally “anything else”

import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [6]:
with open(fname, 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tokens = preprocess(tweet['text'])

In [7]:
#"text":"RT @NFL: .@LukeKuechly Had a Day! #CARvsDAL https:\/\/t.co\/MngpsMwpfz"

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']
terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
print terms_stop

[u'RT', u'@NFL', u'When', u'238', u'lb', u'LB', u'starts', u'making', u'leaping', u'over-the-shoulder', u'grabs', u'like', u'WR', u'#Picked', u'#CARvsDAL', u'#LUUUUUUKE', u'https://t.co/cXqmGIi3', u'\u2026']


In [8]:
''' 
We can further customise the list of terms/tokens we are interested in. For example:
We are interested in only All the Terms, #Terms, Terms without #/@ etc
'''

with open(fname, 'r') as f:
    count_all = Counter()
    count_all_hash = Counter()
    count_all_terms = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text']) if term not in stop]
        # Update the counter
        count_all.update(terms_all)
        # Count terms only once, equivalent to Document Frequency
        terms_single = set(terms_all)
        # Count hashtags only
        terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('#')]
        count_all_hash.update(terms_hash)
        count_all.update(terms_hash)
        # Count terms only (no hashtags, no mentions)
        terms_only = [term for term in preprocess(tweet['text']) if term not in stop and not term.startswith(('#', '@'))] 
        count_all_terms.update(terms_only)
        count_all_terms.update(terms_only)
        
    # Print the first 5 most frequent words
    print(count_all.most_common(10))
    print(count_all_hash.most_common(10))
    print(count_all_terms.most_common(10))

[(u'#CARvsDAL', 44750), (u'RT', 19683), (u'@NFL', 7325), (u'@Panthers', 7047), (u'#Panthers', 6694), (u'Newton', 5199), (u'Dab', 4698), (u'\ud83d', 3909), (u'6', 2988), (u'@dallascowboys', 2848)]
[(u'#CARvsDAL', 22375), (u'#Panthers', 3347), (u'#KeepPounding', 952), (u'#Cowboys', 650), (u'#TOUCHDOWN', 645), (u'#HappyThanksgiving', 574), (u'#NFL', 210), (u'#LUUUUKE', 198), (u'#ColorRush', 197), (u'#LUUUUUUKE', 188)]
[(u'RT', 39366), (u'Newton', 10398), (u'Dab', 9396), (u'\ud83d', 7818), (u'6', 5976), (u'30', 5078), (u'dances', 4672), (u'scores', 4662), (u'https://t.co/mrM8xLdfZh', 4632), (u'0', 4514)]


In [10]:
with open(fname, 'r') as f:
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text']) if term not in stop]

In [11]:
com = defaultdict(lambda : defaultdict(int))
with open(fname, 'r') as f:
    # f is the file pointer to the JSON data set
    for line in f: 
        tweet = json.loads(line)
        terms_only = [term for term in preprocess(tweet['text']) 
                      if term not in stop 
                      and not term.startswith(('#', '@'))]

        # Build co-occurrence matrix
        for i in range(len(terms_only)-1):            
            for j in range(i+1, len(terms_only)):
                w1, w2 = sorted([terms_only[i], terms_only[j]])                
                if w1 != w2:
                    com[w1][w2] += 1

In [12]:
com_max = []
# For each term, look for the most common co-occurrent terms
for t1 in com:
    t1_max_terms = max(com[t1].items(), key=operator.itemgetter(1))[:10]
    for t2 in t1_max_terms:
        com_max.append(((t1, t2), com[t1][t2]))
# Get the most frequent co-occurrences
terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
print(terms_max[:10])

[((u'Dab', u'Newton'), 9318), ((u'Newton', u'RT'), 5057), ((u'\ud83d', u'\udd25'), 4311), ((u'6', u'RT'), 2902), ((u'PANTHERS', u'WIN'), 2539), ((u'30', u'6'), 2530), ((u'dances', u'scores'), 2326), ((u'RT', u'dances'), 2322), ((u'https://t.co/mrM8xLdfZh', u'scores'), 2316), ((u'0', u'11'), 2288)]


In [13]:
search_word = "Newton" # pass a search term
count_search = Counter()
with open(fname, 'r') as f:
    # f is the file pointer to the JSON data set
    for line in f:
        tweet = json.loads(line)
        terms_only = [term for term in preprocess(tweet['text']) if term not in stop and not term.startswith(('#', '@'))]
        if search_word in terms_only:
            count_search.update(terms_only)
print("Co-occurrence for %s:" % search_word)
print(count_search.most_common(20))

Co-occurrence for Newton:
[(u'Newton', 5199), (u'Dab', 4666), (u'RT', 2738), (u'scores', 2327), (u'dances', 2326), (u'https://t.co/mrM8xLdfZh', 2316), (u'Cam', 515), (u'\ud83d', 394), (u'I', 386), (u'left', 365), (u'wave', 363), (u'lot', 361), (u'start', 357), (u'\ude02', 347), (u'people', 329), (u'wanna', 315), (u'would', 42), (u'TD', 40), (u'said', 37), (u'crowd', 33)]
